In [1]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import happybase
import tensorflow as tf
import warnings
import csv
import re
import json
import sklearn.model_selection
import sklearn.preprocessing

In [4]:
data_dir = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"

In [5]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

['/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000000_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000002_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000001_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000003_0']


In [6]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list

b'Skipping line 461777: expected 32 fields, saw 33\n'
<string>:2: DtypeWarning: Columns (1,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 51.9 s, sys: 2.41 s, total: 54.3 s
Wall time: 54.3 s


In [7]:
#
with open("multi_hot_column_odict.pkl","rb") as f:
    multi_hot_column_odict = pickle.load(f)
with open("y_onehot.pkl","rb") as f:
    y_onehot = pickle.load(f)

In [8]:
multi_hot_column_odict.keys()

odict_keys(['关注的公司', '行业'])

In [9]:
model = tf.keras.models.load_model("model_output")

In [10]:
predict_df = df[list(multi_hot_column_odict.keys()) + ["用户id"]]
del df

In [11]:
predict_df

,关注的公司,行业,用户id
0,-1,-1,00001D26FE991A4BC4A01325D5E222DE
1,-1,-1,00006C3EF7F79B0F37EEFB8381505530
2,-1,-1,0000D0E1960239E8362F6B4100D9790A
3,-1,-1,0000f55e60d5ba7b36004429b235c95096f7153d
4,-1,-1,00014E9D5CF948B3FE6376564B23F64F
...,...,...,...
15703260,-1,-1,qlu2xsthqk7tp0mncpgr
15703261,-1,-1,qlu30bysbp1mj150uc1m
15703262,-1,-1,qlu30t7n6f2b1jgzexv
15703263,-1,-1,qlu315b37v5rc273amb0


In [12]:
%%time
#测试数据
warnings.filterwarnings('ignore')
#output_csv = r"/mnt1/zhangwanli/user_profile_output/gender.csv"
output_csv = r"/mnt1/zhangwanli/data/position.csv"
with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['position', 'udid' , "probility"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i,chunk in enumerate(np.array_split(predict_df, 10000)):
        udid_list = chunk["用户id"].values.tolist()
        predict_x = []
        for index in chunk.index:
            temp_x_list = []
            for j,column in enumerate(multi_hot_column_odict):
                split_column_list = chunk.loc[index,column].split(",") if isinstance(chunk.loc[index,column],str) else [str(predict_df.loc[index,column])]
                temp_x_list.extend(multi_hot_column_odict[column].transform([split_column_list]).tolist()[0])
            predict_x.append(temp_x_list)
        y_predict = model.predict([predict_x])
        #print(y_predict)
        for y_predict_i,udid in zip(y_predict[0],udid_list):
            writer.writerow({'position':y_onehot.inverse_transform(np.array([y_predict_i]))[0], 'udid': udid , 'probility':json.dumps(y_predict_i.tolist())})
        if i % 10 == 0:
            print(i)
        #break

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [14]:
import happybase
output_df = pd.read_csv(output_csv)


In [15]:
output_df

,position,udid,probility
0,事业单位员工,00001D26FE991A4BC4A01325D5E222DE,"[0.4527432918548584, 0.10587984323501587, 0.16..."
1,事业单位员工,00006C3EF7F79B0F37EEFB8381505530,"[0.4527432918548584, 0.10587984323501587, 0.16..."
2,事业单位员工,0000D0E1960239E8362F6B4100D9790A,"[0.4527432918548584, 0.10587984323501587, 0.16..."
3,事业单位员工,0000f55e60d5ba7b36004429b235c95096f7153d,"[0.4527432918548584, 0.10587984323501587, 0.16..."
4,事业单位员工,00014E9D5CF948B3FE6376564B23F64F,"[0.4527432918548584, 0.10587984323501587, 0.16..."
...,...,...,...
15703260,事业单位员工,qlu2xsthqk7tp0mncpgr,"[0.4527432918548584, 0.10587984323501587, 0.16..."
15703261,事业单位员工,qlu30bysbp1mj150uc1m,"[0.4527432918548584, 0.10587984323501587, 0.16..."
15703262,事业单位员工,qlu30t7n6f2b1jgzexv,"[0.4527432918548584, 0.10587984323501587, 0.16..."
15703263,事业单位员工,qlu315b37v5rc273amb0,"[0.4527432918548584, 0.10587984323501587, 0.16..."


In [18]:
len(output_df["udid"].unique())

15102652

In [17]:
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(output_df.index):
            batch.put(output_df.loc[index,"udid"].encode(), {"attribute:predict_position".encode(): output_df.loc[index,"position"].encode()})
            #print(output_df.loc[index,"udid"].encode(), {"attribute:predict_position".encode(): output_df.loc[index,"position"].encode()})
            #break
            if i % 100000 == 0:
                print(i)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000